# Setup

In [1]:
import os
import gzip
import json
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from Bio import SeqIO

from IPython.display import display

DIR = r'c://downloads'

plt.style.use('ggplot')

# Q1

Two obvious examples of causal relationships in molecular biology:
1. RNA abundance of transcripts (which is measurable, for example, by RNA-seq) infulences the protein abundance of the same genes (as measurable, for example, by mass spectrometry). This is part of the central dogma of molecular biology.
2. A transcript's length influences the time it would take the ribosome to translate it (the longer the trasncript, the longer we expect the ribosome to work on it).

Two examples of expected non-causal associations:
1. It is expected to observe a positive correlation between the translation time of mRNA molecules to the number of variants  in the human population in those genes, not because either affects the other, but simply because the length of the transcript is likely an important confounder that affects both variables quite dramatically. The longer the transcript, the longer it takes the ribosome to translate it, and the more nucleotides it has that could potentially be mutated.
2. It is expected that the relative (i.e. per-residue) mass and the relative charge of proteins would be correlated, because both are affected by amino-acid composition (as different amino-acids have different mass and charge properties), even though mass and charge do not causally affect one another.

# Q2A

Overall stratrgy:
1. Download all reviewed human proteins from UniProt, and for each record extract: i) its UniProt ID, ii) the sequence of the protein, iii) the locations of all phosphothreonine annotations (i.e. threonine residues that are annotated to undergo, at least sometimes, a phosphorylation PTM).
2. Using the HGNC dataset of human gene names, create a mapping between UniProt to RefSeq IDs, so we end up knowing the RefSeq ID of the records extracted from UniProt.
3. Download all RefSeq transcripts (in the genebank format), and for each of the relevant UniProt records (that we now have its corresponding RefSeq ID), extract the transcript's coding DNA sequence.
4. Filter only proteins that have a perfectly matching CDS sequence.
5. Now that we have both the DNA sequence and locations of phosphorylated threonine residues in each of the filtered human protein, we can count the total number of phosphorylated and unphosphorylated residues for each of the 4 threonine codons. 

In [2]:
# Step 1: Process all reviewed human proteins from UniProt.

def get_phosphothreonine_sites(uniprot_record):
    for feature in record.features:
        if feature.type == 'modified residue' and 'Phosphothreonine' in feature.qualifiers['description']:
            for location in range(feature.location.start, feature.location.end):
                yield location
                
uniprot_ids = []
protein_seqs = []
phospho_sites = []

# From: http://www.uniprot.org/uniprot/?query=organism%3A%22Homo+sapiens+%5B9606%5D%22+AND+reviewed%3Ayes&sort=score
with gzip.open(os.path.join(DIR, 'uniprot_human_reviewed.xml.gz'), 'r') as f:
    for record in SeqIO.parse(f, 'uniprot-xml'):
        uniprot_ids.append(record.id)
        protein_seqs.append(record.seq)
        phospho_sites.append(list(get_phosphothreonine_sites(record)))
            
uniprot_data = pd.DataFrame({'uniprot_id': uniprot_ids, 'protein_seq': protein_seqs, 'phospho_sites': phospho_sites})
display(uniprot_data)

,uniprot_id,protein_seq,phospho_sites
0,Q9Y263,"(M, T, S, G, A, T, R, Y, R, L, S, C, S, L, R, ...",[]
1,Q96RE7,"(M, A, Q, T, L, Q, M, E, I, P, N, F, G, N, S, ...",[]
2,O43312,"(M, E, A, V, I, E, K, E, C, S, A, L, G, G, L, ...","[257, 424, 602]"
3,Q9NP80,"(M, S, I, N, L, T, V, D, I, Y, I, Y, L, L, S, ...",[]
4,Q15319,"(M, M, A, M, N, S, K, Q, P, F, G, M, H, P, V, ...",[]
5,P08134,"(M, A, A, I, R, K, K, L, V, I, V, G, D, G, A, ...",[]
6,Q5VZM2,"(M, E, E, S, D, S, E, K, T, T, E, K, E, N, L, ...",[]
7,P20338,"(M, S, Q, T, A, M, S, E, T, Y, D, F, L, F, K, ...",[]
8,P61020,"(M, T, S, R, S, T, A, R, P, N, G, Q, P, Q, A, ...",[]
9,Q9BRS2,"(M, D, Y, R, R, L, L, M, S, R, V, V, P, G, Q, ...",[]


In [3]:
# Step 2: Create a mapping between UniProt and RefSeq IDs by processing the genenames annotations.

refseqs = []
uniprot_ids = []

# From: ftp://ftp.ebi.ac.uk/pub/databases/genenames/new/json/non_alt_loci_set.json
with open(os.path.join(DIR, 'non_alt_loci_set.json'), 'r', encoding = 'utf-8') as f:
    gene_names = json.load(f)

for record in gene_names['response']['docs']:
    for refseq in record.get('refseq_accession', []):
        for uniprot_id in record.get('uniprot_ids', []):
            refseqs.append(refseq)
            uniprot_ids.append(uniprot_id)
        
uniprot_and_refseqs = pd.DataFrame({'uniprot_id': uniprot_ids, 'refseq': refseqs})
display(uniprot_and_refseqs)

,uniprot_id,refseq
0,P04217,NM_130786
1,Q9NQ94,NM_014576
2,P01023,NM_000014
3,A8K2U0,NM_144670
4,U3KPV4,NM_001080438
5,Q9NPC4,NM_017436
6,Q9UNA3,NM_016161
7,Q9NRG9,NM_001173466
8,Q86V21,NM_023928
9,P22760,NM_001086


In [4]:
# Merging the RefSeq IDs into the processed UniProt records.
uniprot_data = pd.merge(uniprot_and_refseqs, uniprot_data, on = 'uniprot_id')
display(uniprot_data)

,uniprot_id,refseq,protein_seq,phospho_sites
0,P04217,NM_130786,"(M, S, M, L, V, V, F, L, L, L, W, G, V, T, W, ...",[]
1,Q9NQ94,NM_014576,"(M, E, S, N, H, K, S, G, D, G, L, S, G, T, Q, ...",[498]
2,P01023,NM_000014,"(M, G, K, N, K, L, L, H, P, S, L, V, L, L, L, ...",[]
3,A8K2U0,NM_144670,"(M, W, A, Q, L, L, L, G, M, L, A, L, S, P, A, ...",[]
4,U3KPV4,NM_001080438,"(M, A, L, K, E, G, L, R, A, W, K, R, I, F, W, ...",[]
5,Q9NPC4,NM_017436,"(M, S, K, P, P, D, L, L, L, R, L, L, R, G, A, ...",[]
6,Q9UNA3,NM_016161,"(M, R, K, E, L, Q, L, S, L, S, V, T, L, L, L, ...",[]
7,Q9NRG9,NM_001173466,"(M, C, S, L, G, L, F, P, P, P, P, P, R, G, Q, ...",[]
8,Q86V21,NM_023928,"(M, S, K, E, E, R, P, G, R, E, E, I, L, E, C, ...",[]
9,P22760,NM_001086,"(M, G, R, K, S, L, Y, L, L, I, V, G, I, L, I, ...",[]


In [5]:
# Step 3: Like in HW Exercise 4, download the entire human transcriptome in genebank fromat from RefSeq's FTP site at:
# ftp://ftp.ncbi.nlm.nih.gov/refseq/H_sapiens/mRNA_Prot/ 
# (i.e. all files of the form: human.*.rna.gbff.gz)
# Parse only relevant records (with a RefSeq ID of one of the processed records). Use the CDS feature to get the start
# and end coordinates of each transcript's coding region, so the relevant coding sequence can be extracted.

def get_features_of_type(features, type_name):
    return [feature for feature in features if feature.type == type_name]

relevant_refseq_ids = set(uniprot_data['refseq'].unique())
refseq_id_to_cds_seq = {}

for file_index in range(7):

    print('Analyzing file #%d...' % (file_index + 1))

    with gzip.open(os.path.join(DIR, 'human.%d.rna.gbff.gz' % (file_index + 1)), 'rt') as f:
        for record in SeqIO.parse(f, 'genbank'):
        
            refseq_id = record.id.split('.')[0]
        
            if refseq_id in relevant_refseq_ids:
                
                cds_features = [feature for feature in record.features if feature.type == 'CDS']
                
                if len(cds_features) > 0:
                    cds_feature, = cds_features
                    assert refseq_id not in refseq_id_to_cds_seq, 'Duplicate RefSeq ID: %s' % refseq_id
                    refseq_id_to_cds_seq[refseq_id] = record.seq[cds_feature.location.start:cds_feature.location.end]

Analyzing file #1...
Analyzing file #2...
Analyzing file #3...
Analyzing file #4...
Analyzing file #5...
Analyzing file #6...
Analyzing file #7...


In [6]:
# Step 4: Set the CDS sequences of the protien records, and keep only records with perfectly matching sequences.

uniprot_data['cds_seq'] =  uniprot_data['refseq'].map(refseq_id_to_cds_seq)
cds_seq_mask = pd.notnull(uniprot_data['cds_seq'])
print('%d of %d protein records don\'t have a CDS DNA sequence. Filtering them out.' % ((~cds_seq_mask).sum(), \
        len(cds_seq_mask)))
uniprot_data = uniprot_data[cds_seq_mask]

matching_seqs_mask = uniprot_data.apply(lambda record: record['protein_seq'] == record['cds_seq'].translate()[:-1], axis = 1)
print('%d of %d protein records have a CDS DNA sequence not matching their protein sequence. Filtering them out.' % 
        ((~matching_seqs_mask).sum(), len(matching_seqs_mask)))
uniprot_data = uniprot_data[matching_seqs_mask]

display(uniprot_data)

1430 of 20255 protein records don't have a CDS DNA sequence. Filtering them out.


C:\Users\Nadav\Anaconda2\envs\py37\lib\site-packages\Bio\Seq.py:2576: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning)


3362 of 18825 protein records have a CDS DNA sequence not matching their protein sequence. Filtering them out.


,uniprot_id,refseq,protein_seq,phospho_sites,cds_seq
0,P04217,NM_130786,"(M, S, M, L, V, V, F, L, L, L, W, G, V, T, W, ...",[],"(A, T, G, T, C, C, A, T, G, C, T, C, G, T, G, ..."
2,P01023,NM_000014,"(M, G, K, N, K, L, L, H, P, S, L, V, L, L, L, ...",[],"(A, T, G, G, G, G, A, A, G, A, A, C, A, A, A, ..."
3,A8K2U0,NM_144670,"(M, W, A, Q, L, L, L, G, M, L, A, L, S, P, A, ...",[],"(A, T, G, T, G, G, G, C, T, C, A, G, C, T, C, ..."
4,U3KPV4,NM_001080438,"(M, A, L, K, E, G, L, R, A, W, K, R, I, F, W, ...",[],"(A, T, G, G, C, T, C, T, C, A, A, G, G, A, G, ..."
5,Q9NPC4,NM_017436,"(M, S, K, P, P, D, L, L, L, R, L, L, R, G, A, ...",[],"(A, T, G, T, C, C, A, A, G, C, C, C, C, C, C, ..."
6,Q9UNA3,NM_016161,"(M, R, K, E, L, Q, L, S, L, S, V, T, L, L, L, ...",[],"(A, T, G, C, G, G, A, A, G, G, A, G, C, T, C, ..."
8,Q86V21,NM_023928,"(M, S, K, E, E, R, P, G, R, E, E, I, L, E, C, ...",[],"(A, T, G, T, C, C, A, A, G, G, A, G, G, A, G, ..."
9,P22760,NM_001086,"(M, G, R, K, S, L, Y, L, L, I, V, G, I, L, I, ...",[],"(A, T, G, G, G, A, A, G, A, A, A, A, T, C, G, ..."
10,Q6P093,NM_207365,"(M, G, L, K, A, L, C, L, G, L, L, C, V, L, F, ...",[],"(A, T, G, G, G, G, C, T, A, A, A, A, G, C, T, ..."
11,Q5VUY0,NM_001103170,"(M, W, D, L, A, L, I, F, L, A, A, A, C, V, F, ...",[],"(A, T, G, T, G, G, G, A, C, C, T, G, G, C, C, ..."


In [7]:
# Step 5: Using the constructed dataset, count the total number of threonine codons in the human transcriptome, and how
# many of them are phosphorylated.

all_threonine_codons = Counter()
phosphothreonine_codons = Counter()

for _, record in uniprot_data.iterrows():
    for i, aa in enumerate(str(record['protein_seq'])):
        if aa == 'T':
            
            codon = str(record['cds_seq'][(3 * i):(3 * (i + 1))])
            all_threonine_codons[codon] += 1
            
            if i in record['phospho_sites']:
                phosphothreonine_codons[codon] += 1
                
codon_counts = pd.DataFrame({'all': all_threonine_codons, 'phospho': phosphothreonine_codons})
codon_counts['phospho_freq'] = codon_counts['phospho'] / codon_counts['all']
codon_counts.sort_index(inplace = True)
display(codon_counts)

,all,phospho,phospho_freq
ACA,127451,1248,0.009792
ACC,155909,1419,0.009101
ACG,50365,566,0.011238
ACT,113694,1180,0.010379


It appears that ACG codons are more likely (with probability of 1.12%) to code for threonine residues undergoing phosphorylation. The next codon most likely to undergo phosphorylation is ACT, with probability of 1.04%. ACA and ACC are less likely to undergo phosphrylation, with probabilities of 0.98% and and 0.91%, respectively. 

# Q2B

In [8]:
pospho_counts = codon_counts['phospho'].rename('Phosphorylated amino-acid')
no_pospho_counts = (codon_counts['all'] - pospho_counts).rename('Not phosphorylated')
exclusive_counts = pd.concat([pospho_counts, no_pospho_counts], axis = 1)

ACG_contingency_table = pd.DataFrame([exclusive_counts.loc['ACG'].rename('ACG codon'), \
        exclusive_counts.loc[[codon for codon in exclusive_counts.index if codon != 'ACG']].sum().rename('Other codons')])
display(ACG_contingency_table)

,Phosphorylated amino-acid,Not phosphorylated
ACG codon,566,49799
Other codons,3847,393207


In [9]:
from scipy.stats import fisher_exact

print('Fisher\'s exact test results for the enrichment of the ACG codon with phosphothreonine: OR = %.2f, p-value = %.2g' % \
        fisher_exact(ACG_contingency_table))

Fisher's exact test results for the enrichment of the ACG codon with phosphothreonine: OR = 1.16, p-value = 0.0011


We observe a mild, but apparently significant correlation between the ACG threonine codon to phosphorylation, compared to the other three codons (OR = 1.16, p-value = 0.0011). Therefore, it seems reasonable to conclude that ACG-codon threonine residues are in fact more likely to undergo phosphorylation compared to other codons.

# Q2C

There are many technical and confounding factors that could potentially explain the observed association of the ACG codon with posphothreonine. For example:
1. UniProt's PTM annotations are far from comprehensive, and are heavily biased by how much attention different proteins have received from the scientific community. It is very plausible that certain groups of proteins that are characterized by various factors (e.g. relevance to human health or defined pathways) have been more thoroughly studied, and as a result contain more PTM annotations (and, in particular, phosphorylation annotations). It is also not unreasonable to think that these groups of proteins could be unrepresentative in their DNA sequence, and in particular in their codon usage (e.g. maybe they are overrepresented by certain domain families that, for evolutionary reasons, contain more ACG codons). If that's the case, we will end up with ACG-rich proteins being also enriched, for technical reasons, with phosphorylation annotations.
2. It is possible that this association is driven by only a handful of protien or domain families. For example, maybe there are families of ACG-rich proteins that, for completely other reasons, also undergo more phosphorylation than average.
3. Another possible confounder could be the locatin of the residues on the protein sequence (either on the 1D sequence, or the 3D structure). For example, phosphorylation resiudes must be exposed (i.e. on the surface of the protein). Likewise, there could also be enrichment of phosphorylated residues at the beginning or end of protein seqeunces, and maybe, for totally other reasons, those locations are also enriched with ACG codons.

A hypothetical biological mechanism that could explain a causal relationship between ACG codons to posphorylation is folding dynamics. It is known that different synonymous codons (all coding for the same amino-acid) could lead to different translation speeds by the ribosome (e.g. due to differenes in tRNA abundance). Translation dynamics might also, in certain cases, influence a protein's folding. Protein folding, in turn, can affect its phosphorylation pattern (e.g. by influencing which residues are exposed and which are burried).

Of course, all mentioned "explanations" are highly speculative. The bottom line is that it's very difficult to come up with one true, established explanation given just a simple correlation.